# 数据集目录结构
```
FashionAI-Attributes
├── models/
├── data/
│   ├── base/
│   ├── rank/
│   └── web/
└── solution.ipynb
```

In [1]:
import os
import cv2
import random
import re
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from PIL import Image

%matplotlib inline

In [ ]:
image_demo = cv2.imread('./data/base/Images/coat_length_labels/fff3f9da02b33c0d2619a1dde0914737.jpg')
image_demo.shape

In [2]:
df_train = pd.read_csv('./data/base/Annotations/label.csv', header=None)
df_train.columns = ['image_id', 'class', 'label']
df_train.head()

,image_id,class,label
0,Images/collar_design_labels/4d8a38b29930a403e5...,collar_design_labels,nnynn
1,Images/collar_design_labels/bd0981f231180d2b00...,collar_design_labels,nynnn
2,Images/collar_design_labels/26937e1724feadfe39...,collar_design_labels,ynnnn
3,Images/collar_design_labels/cf4140ec542887270f...,collar_design_labels,nynnn
4,Images/collar_design_labels/50644b2b9de045f2d1...,collar_design_labels,nynnn


In [3]:
classes = ['collar_design_labels', 'neckline_design_labels', 'skirt_length_labels', 
           'sleeve_length_labels', 'neck_design_labels', 'coat_length_labels', 'lapel_design_labels', 
           'pant_length_labels']

In [4]:
cur_class = classes[0]
df_load = df_train[(df_train['class'] == cur_class)].copy
#df_load = df_train[(df_train['class'] == cur_class)].head(1000)
df_load.reset_index(inplace=True)
del df_load['index']

print('{0}: {1}'.format(cur_class, len(df_load)))
df_load.head()

collar_design_labels: 1000


,image_id,class,label
0,Images/collar_design_labels/4d8a38b29930a403e5...,collar_design_labels,nnynn
1,Images/collar_design_labels/bd0981f231180d2b00...,collar_design_labels,nynnn
2,Images/collar_design_labels/26937e1724feadfe39...,collar_design_labels,ynnnn
3,Images/collar_design_labels/cf4140ec542887270f...,collar_design_labels,nynnn
4,Images/collar_design_labels/50644b2b9de045f2d1...,collar_design_labels,nynnn


In [5]:
width = 224
n = len(df_load)
n_class = len(df_load['label'][0])

X = np.zeros((n, width, width, 3), dtype='float32')
Y = np.zeros((n, n_class), dtype='float32')

In [6]:
for i in range(n):
    tmp_label = df_load['label'][i]
    if len(tmp_label) > n_class:
        print(df_load['image_id'][i])
    X[i] = cv2.resize(cv2.imread('./data/base/{0}'.format(df_load['image_id'][i])), (width, width))/255.
    Y[i][tmp_label.find('y')] = 1
    if i == 100:
        print(X[i])
        print(Y[i])
        print(tmp_label)

[[[0.5647059  0.53333336 0.53333336]
  [0.5686275  0.5372549  0.5372549 ]
  [0.5647059  0.5372549  0.5372549 ]
  ...
  [0.46666667 0.4392157  0.42745098]
  [0.45882353 0.43137255 0.41960785]
  [0.4627451  0.43529412 0.41960785]]

 [[0.5647059  0.53333336 0.53333336]
  [0.5686275  0.5372549  0.5372549 ]
  [0.57254905 0.5411765  0.5411765 ]
  ...
  [0.4627451  0.43529412 0.42352942]
  [0.45490196 0.42745098 0.41568628]
  [0.45882353 0.43137255 0.41960785]]

 [[0.5686275  0.5372549  0.5372549 ]
  [0.5686275  0.5372549  0.5372549 ]
  [0.5686275  0.5372549  0.5372549 ]
  ...
  [0.4745098  0.4392157  0.42745098]
  [0.4745098  0.4392157  0.42745098]
  [0.4745098  0.43529412 0.42745098]]

 ...

 [[0.81960785 0.7921569  0.75686276]
  [0.8235294  0.79607844 0.7607843 ]
  [0.81960785 0.7921569  0.75686276]
  ...
  [0.7372549  0.70980394 0.6745098 ]
  [0.7372549  0.70980394 0.6745098 ]
  [0.7411765  0.7137255  0.6784314 ]]

 [[0.827451   0.8        0.7647059 ]
  [0.8235294  0.79607844 0.7607843 ]


In [22]:
from keras import models
from keras import layers
from keras import optimizers
from keras.applications import ResNet50, VGG16
from keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [23]:
conv_base = ResNet50(weights='imagenet', include_top=False, input_shape=(width, width, 3))
# conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(width, width, 3))

In [24]:
conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [10]:
for layer in conv_base.layers:
    if re.search(r'^res5', layer.name) is not None:
        layer.trainable = True
    else:
         layer.trainable = False
for layer in conv_base.layers:
    print(layer, layer.trainable)

<keras.engine.topology.InputLayer object at 0x14cd86400> False
<keras.layers.convolutional.ZeroPadding2D object at 0x14cd86588> False
<keras.layers.convolutional.Conv2D object at 0x14cd86518> False
<keras.layers.normalization.BatchNormalization object at 0x11ba595c0> False
<keras.layers.core.Activation object at 0x14cd869e8> False
<keras.layers.pooling.MaxPooling2D object at 0x14ce193c8> False
<keras.layers.convolutional.Conv2D object at 0x14cdb1198> False
<keras.layers.normalization.BatchNormalization object at 0x14ce2e828> False
<keras.layers.core.Activation object at 0x14cf04c50> False
<keras.layers.convolutional.Conv2D object at 0x14cf1c908> False
<keras.layers.normalization.BatchNormalization object at 0x14cf1c6d8> False
<keras.layers.core.Activation object at 0x14cf9a6d8> False
<keras.layers.convolutional.Conv2D object at 0x14d0588d0> False
<keras.layers.convolutional.Conv2D object at 0x14d0da6a0> False
<keras.layers.normalization.BatchNormalization object at 0x14d0586a0> False
<

In [25]:
model = models.Sequential() 
model.add(conv_base) 
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(n_class, activation='softmax'))

In [26]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
global_average_pooling2d_2 ( (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 2565      
Total params: 14,979,909
Trainable params: 14,979,909
Non-trainable params: 0
_________________________________________________________________


In [28]:
X_train,X_test, y_train, y_test = train_test_split(X, Y, test_size=0.10, random_state=42)

In [29]:
print(X_train.shape)
print(X_test.shape)
print("Length of X_train: " + str(len(X_train)))
print("Length of X_test: " + str(len(X_test)))

(900, 224, 224, 3)
(100, 224, 224, 3)
Length of X_train: 900
Length of X_test: 100


In [32]:
prefix_cls = cur_class.split('_')[0]

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

checkpoint = ModelCheckpoint(filepath='./models/{0}.best.h5'.format(prefix_cls), monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early = EarlyStopping(monitor="val_loss", mode="min", patience=20)

callbacks_list = [checkpoint, early] #early
history = model.fit(
    X_train, 
    y_train, 
    batch_size=int(810/32), 
    epochs=10,
    callbacks=callbacks_list,
    shuffle=True, 
    validation_split=0.1)

Train on 810 samples, validate on 90 samples
Epoch 1/10
 50/810 [>.............................] - ETA: 28:54 - loss: 1.6792 - acc: 0.2400

KeyboardInterrupt: 

In [18]:
model.evaluate(X_test, y_test)

100/100 [==============================] - 36s 357ms/step


[1.9702762794494628, 0.15]

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc'] 
val_acc = history.history['val_acc'] 
loss = history.history['loss'] 
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc') 
plt.plot(epochs, val_acc, 'b', label='Validation acc') 
plt.title('Training and validation accuracy') 
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss') 
plt.plot(epochs, val_loss, 'b', label='Validation loss') 
plt.title('Training and validation loss') 
plt.legend()

plt.show()

In [21]:
from keras.preprocessing import image


def load_image(img_path, show=False):

    img = image.load_img(img_path, target_size=(224, 224))
    img_tensor = image.img_to_array(img)                    # (height, width, channels)
    img_tensor = np.expand_dims(img_tensor, axis=0)         # (1, height, width, channels), add a dimension because the model expects this shape: (batch_size, height, width, channels)
    img_tensor /= 255.                                      # imshow expects values in the range [0, 1]

    if show:
        plt.imshow(img_tensor[0])                           
        plt.axis('off')
        plt.show()

    return img_tensor

df_new = df_train[(df_train['class'] == cur_class)].head(2000)
image_id = df_new['image_id'][1200]
new_image_label = df_new['label'][1200]

img_path = os.path.join("./data/base/", image_id)
new_image = load_image(img_path)
pred = model.predict(new_image) 
print(pred)
print(new_image_label)

[[0.11388171 0.25267315 0.4675808  0.04555008 0.12031426]]
nnynn
